### Calculate returns across all strike prices for all strategies
#### Option Returns Data Elements
- option_strategy
- entry_date
- moneyness_buy_to_open_call
- moneyness_sell_to_open_call
- moneyness_buy_to_open_put
- moneyness_sell_to_open_put
- pct_return
- market_direction
- sampling_key

Moneyness refers to the probability that the strike price that was bought or sold will close in the money on expiry date.
<br/>
 Each strategy will have its own output file and only the moneyness fields relevent to the strategy will be saved

#### The following csv files will be created
- buy_call_returns
- buy_put_returns
- buy_call_spread_returns
- buy_put_spread_returns
- sell_call_spread_returns
- sell_put_spread_returns



In [20]:
import pandas as pd
import csv
import datetime

#### Strategy: Buy Call

In [48]:
def buy_call_return(entry_call_ask, exit_call_price):
    """calculate percentage return"""
    if exit_call_price == 0:
        return -100.0
    pct_return = round((exit_call_price - entry_call_ask) / entry_call_ask * 100, 0)
    return pct_return


In [50]:
def buy_call():
    """
    Calculate returns for Buy Call Strategy
    """
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    spy_1wk_options['pct_return'] = spy_1wk_options.apply(lambda x: buy_call_return(x['entry_call_ask'], x['exit_call_price']), axis=1)
    
    with open('data/buy_call_returns.csv', 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'moneyness_buy_to_open_call', 'pct_return', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        for row in spy_1wk_options.itertuples():
            record = [
                datetime.datetime.date(row.entry_date),
                round(row.entry_call_moneyness, 3),
                row.pct_return,
                row.market_direction,
                row.sampling_key]
            out_csv.writerow(record)      
    
#run function
buy_call()
    

#### Strategy: Buy Put













In [52]:
def buy_put_return(entry_put_ask, exit_put_price):
    """calculate percentage return"""
    if exit_put_price == 0:
        return -100.0
    pct_return = round((exit_put_price - entry_put_ask) / entry_put_ask * 100, 0)
    return pct_return

In [53]:
def buy_put():
    """
    Calculate returns for Buy Put Strategy    
    """
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    spy_1wk_options['pct_return'] = spy_1wk_options.apply(lambda x: buy_put_return(x['entry_put_ask'], x['exit_put_price']), axis=1)
    
    with open('data/buy_put_returns.csv', 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'moneyness_buy_to_open_put', 'pct_return', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        for row in spy_1wk_options.itertuples():
            record = [
                datetime.datetime.date(row.entry_date),
                round(row.entry_put_moneyness, 3),
                row.pct_return,
                row.market_direction,
                row.sampling_key]
            out_csv.writerow(record)      
    
#run function
buy_put()

#### Strategy: Buy Call Spread

In [61]:
def buy_call_spread_return(entry_buy_strike_ask, entry_sell_strike_bid, exit_buy_strike_price, exit_sell_strike_price):
    """calculate percentage return"""
    if exit_buy_strike_price == 0:
        return -100.0
    spread_cost = entry_buy_strike_ask - entry_sell_strike_bid
    spread_proceeds = exit_buy_strike_price - exit_sell_strike_price
    pct_return = round((spread_proceeds - spread_cost) / spread_cost * 100, 0)
    return pct_return

In [88]:
def buy_call_spread():
    """
    Calculate returns for Buy Call Spread Strategy
    """
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    entry_dates = spy_1wk_options['entry_date'].unique()
    
    with open('data/buy_call_spread_returns.csv', 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'moneyness_buy_to_open_call', 'moneyness_sell_to_open_call',
            'pct_return', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
    
        for entry_date in entry_dates:
            option_set = spy_1wk_options[spy_1wk_options['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike price record with every other strike price record
            for i in range(row_count):
                for j in range(i + 1, row_count):
                    buy_strike_record = option_set.iloc[i]
                    sell_strike_record = option_set.iloc[j]
                    
                    pct_return = buy_call_spread_return(
                            buy_strike_record['entry_call_ask'],
                            sell_strike_record['entry_call_bid'],
                            buy_strike_record['exit_call_price'],
                            sell_strike_record['exit_call_price'])

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['entry_call_moneyness'], 3),
                        round(sell_strike_record['entry_call_moneyness'], 3),
                        pct_return,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)      

 #run function
buy_call_spread()
                    

In [89]:
test = pd.read_csv('data/buy_call_spread_returns.csv')
test.head()

,entry_date,moneyness_buy_to_open_call,moneyness_sell_to_open_call,pct_return,market_direction,sampling_key
0,2015-07-02,0.949,0.938,-66.0,Neutral Up,0.838
1,2015-07-02,0.949,0.926,-49.0,Neutral Up,0.838
2,2015-07-02,0.949,0.912,-35.0,Neutral Up,0.838
3,2015-07-02,0.949,0.896,-24.0,Neutral Up,0.838
4,2015-07-02,0.949,0.878,-23.0,Neutral Up,0.838


In [91]:
test.tail()

,entry_date,moneyness_buy_to_open_call,moneyness_sell_to_open_call,pct_return,market_direction,sampling_key
97315,2020-06-19,0.084,0.064,-100.0,Neutral Down,0.092
97316,2020-06-19,0.084,0.055,-100.0,Neutral Down,0.092
97317,2020-06-19,0.073,0.064,-100.0,Neutral Down,0.092
97318,2020-06-19,0.073,0.055,-100.0,Neutral Down,0.092
97319,2020-06-19,0.064,0.055,-100.0,Neutral Down,0.092


In [81]:
byte_size = 49244463
byte_size / 2 ** 20


46.96317958831787